In [1]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 17.8 MB/s eta 0:00:00


In [2]:
!pip install streamlit
!pip install langchain
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -q streamlit
!pip install pyngrok
!pip install langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-o8vm_jpx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-o8vm_jpx
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=80355

In [3]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings

In [11]:
!ngrok authtoken 2p2H1ugnBcbHJLUPOLtYTg44mwW_x4J65Yfo3T9MPBF4LCAV

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
!ngrok

ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions

In [13]:
%%writefile app.py
import streamlit as st
import os

import fitz  # PyMuPDF
import datetime as dt
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

# Function to load and process PDF
def load_pdf(uploaded_file):
    """Loads a PDF file and extracts its text content.

    Args:
        uploaded_file: The uploaded file object from Streamlit.

    Returns:
        A string containing the entire text content of the PDF.
    """
    pdf_text = []

    # Use uploaded_file.read() to get the file content
    with fitz.open(stream=uploaded_file.read(), filetype="pdf") as pdf:
        for page in pdf:
            pdf_text.append(page.get_text())
    return " ".join(pdf_text)

def process_text(pdf_text):
    texts = pdf_text.split(". ")
    grouped_texts, time_list = [], []
    for i, text in enumerate(texts):
        time_list.append(dt.datetime.now() + dt.timedelta(seconds=i*30))  # Set pseudo-timestamps for grouping
        grouped_texts.append(text)

    return grouped_texts, time_list

def get_vectorstore(grouped_texts, time_list):
    embeddings = HuggingFaceEmbeddings()
    time_stamps = [{'source': str(t)} for t in time_list]
    vectorstore = FAISS.from_texts(texts=grouped_texts, embedding=embeddings, metadatas=time_stamps)
    return vectorstore

def get_conversation(vectorstore):
    with open('token.txt', 'r') as f:
        token = f.read().strip()

    llm = HuggingFaceHub(
        repo_id="google/flan-t5-base",
        model_kwargs={"temperature": 0.5, "max_length": 512},
        huggingfacehub_api_token=token
    )
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        memory=memory,
        retriever=vectorstore.as_retriever()
    )
    return conversation_chain

def main():
    st.set_page_config(page_title="Chat with PDF", page_icon=":books:")
    st.header("Chat with your PDF Document 📄")

    uploaded_file = st.sidebar.file_uploader("Upload your PDF", type="pdf")

    if uploaded_file is not None:
        pdf_text = load_pdf(uploaded_file)
        grouped_texts, time_list = process_text(pdf_text)
        vectorstore = get_vectorstore(grouped_texts, time_list)

        st.write("Now you can start chatting!")

        user_question = st.text_input("Enter your question here:")
        if user_question:
            chain = get_conversation(vectorstore)
            response = chain({"question": user_question, "chat_history": []})['answer']
            st.write(response)

if __name__ == "__main__":
    main()


Overwriting app.py


In [14]:
!streamlit run app.py &>/dev/null&

In [15]:
!pgrep streamlit

1482
2240


In [16]:
!pip install pyngrok
from pyngrok import ngrok

# Attempt to kill any existing ngrok processes
try:
    ngrok.kill()
except Exception as e:
    print(f"Error killing existing process: {e}")
    print("Attempting to kill manually...")
    !pkill -f ngrok  # Manually kill any process with "ngrok" in its name

# Now you can connect to port 8501
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "https://9cf7-34-66-47-131.ngrok-free.app" -> "http://localhost:8501">